In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/2024_Unstructured_Data/

/content/drive/MyDrive/2024_Unstructured_Data


# environment setting

In [ ]:
!pip install -q -U "torch==2.1.2" tensorboard
!pip install -q -U "transformers==4.36.2" "datasets==2.16.1" "accelerate==0.26.1" "bitsandbytes==0.42.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 44.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires tensorboard<2.16,>=2.15, but you have tensorboard 2.17.0 which is incompatible.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.1.2 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 2.1.2 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.1.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -U "huggingface_hub[cli]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.23.2
    Uninstalling huggingface-hub-0.23.2:
      Successfully uninstalled huggingface-hub-0.23.2


In [ ]:
!huggingface-cli login --token ""

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Library Import & Load Model

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Load Dataset & Prepare Training

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# df_miss = pd.read_csv("./data/depression_missclassification_cases.csv")
df_miss = pd.read_csv("./data/mixed_emotion_set_missclassification_cases.csv")
df_miss = df_miss[df_miss["label"] != "Neutral"].reset_index(drop=True)
df_miss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   text               4 non-null      object 
 1   label              4 non-null      object 
 2   prediction         4 non-null      object 
 3   max_prob           4 non-null      float64
 4   prob_distribution  4 non-null      object 
dtypes: float64(1), object(4)
memory usage: 288.0+ bytes


In [ ]:
df_miss.head()

,text,label,prediction,max_prob,prob_distribution
0,I am a housewife in my 20s. Through early marr...,Depression,Happy,0.903991,"[0.09373518079519272, 0.002274321625009179, 0...."
1,Despite my successful career and the admiratio...,Depression,Happy,0.576257,"[0.41973501443862915, 0.004007911309599876, 0...."
2,Mom and Dad had another big fight about who ge...,Depression,Happy,0.609991,"[0.38730886578559875, 0.0027002866845577955, 0..."
3,It's been a challenging time for me lately. Th...,Happy,Depression,0.912226,"[0.9122256636619568, 0.0021298956125974655, 0...."


In [ ]:
X_train, y_train = df_miss["text"], df_miss["prediction"]

# LLM Load

In [ ]:
def load_LLM_huggingface(model_name, device, compute_dtype):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name,
                                                  trust_remote_code=True,
                                                  )
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=compute_dtype,
        device_map=device,
        quantization_config=bnb_config,
    )
    return [tokenizer, model]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
models = ["meta-llama/Llama-2-7b-chat-hf", "meta-llama/Meta-Llama-3-8B-Instruct"] # Need login
compute_dtype = getattr(torch, "float16")

# LLaMA2 = load_LLM_huggingface(models[0], device, compute_dtype)
LLaMA3 = load_LLM_huggingface(models[1], device, compute_dtype)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

# Inference

In [ ]:
from tqdm.auto import tqdm
from prompts import(
    select_prompt,
    reasoning_modules,
    adapt_prompt,
    implement_prompt
)

In [ ]:
def Self_Discover(model, data, label, task):
    def reply(prompt):
        messages = [
            {"role": "user", "content": prompt},
        ]
        input_ids = model[0].apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model[1].device)

        terminators = [
            model[0].eos_token_id,
            model[0].convert_tokens_to_ids("<|eot_id|>")
        ]

        outputs = model[1].generate(
            input_ids,
            max_new_tokens=2048,
            eos_token_id=terminators,
            do_sample=True,
            top_p=1,
            pad_token_id=model[0].eos_token_id
        )
        response = outputs[0][input_ids.shape[-1]:]
        return model[0].decode(response, skip_special_tokens=True)
    # SELECT
    prompt = select_prompt.replace("{Task}", task)
    prompt = prompt.replace("{resonining_modules}", reasoning_modules)
    selected_modules = reply(prompt)

    # ADAPT
    prompt = adapt_prompt.replace("{Task}", task)
    prompt = prompt.replace("{selected_modules}", selected_modules)
    adapted_modules = reply(prompt)

    # IMPLEMENT
    prompt = implement_prompt.replace("{Task}", task)
    prompt = prompt.replace("{adapted_modules}", adapted_modules)
    reasoning_structure = reply(prompt)

    return selected_modules, adapted_modules, reasoning_structure

In [ ]:
def Stage2(model, data, label, task, reasoning_structure):
    def reply(prompt):
        messages = [
            {"role": "system", "content": "You are a professional psychologist and advisor, and we expect you to comply with our demands."},
            # {"role": "system", "content": "As a professional psychologist and advisor, you look at the data we present to you and fill out the survey."},
            {"role": "user", "content": prompt},
        ]
        input_ids = model[0].apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model[1].device)

        terminators = [
            model[0].eos_token_id,
            model[0].convert_tokens_to_ids("<|eot_id|>")
        ]

        outputs = model[1].generate(
            input_ids,
            max_new_tokens=2048,
            eos_token_id=terminators,
            do_sample=False,
            # temperature = 1.0,
            # temperature = 0.1,
            # top_p=1,
            pad_token_id=model[0].eos_token_id
        )
        response = outputs[0][input_ids.shape[-1]:]
        return model[0].decode(response, skip_special_tokens=True)
    prompt = f"Using the following reasoning structure: {reasoning_structure}\n\nSolve this task, providing your answer: {task_}\n\nNote1:  Write the question number before your answer.\nNote2: Don't write anything besides your answer."
    answer = reply(prompt)
    return answer

In [ ]:
task = """
I will give you a text data. Your job is to give me logical reasoning of following questions within a given texts.
That given data is text, and AI-derived classified label about emotion.
The text data contains one or more emotions such as "Depression", "Neutral", and "Happy".

<context>

data: {data}

result: {label}

<questions>

1. Objectively analyze given data.
2. Compare the analysis results from Question 1 to AI's past classified label.
3. Synthesize results from Questions 1 and 2 and write a final result for the given data.
4. Choose only one of the following that most closely resembles your analytics results. Don't create or explain additional options beyond the choices.
<Options>"Depression", "Neutral", "Happy", "Mixed Emotions"</Options>

</questions>
"""

In [ ]:
# a=[]
b=[]
# c=[]
d=[]
for X, y in tqdm(zip(X_train.values, y_train), desc = "Generate Reasoning"):
    task_ = task.replace("{data}", X).replace("{label}", y)
    # a.append(Self_Discover(LLaMA2, X, y, task_))
    # c.append(Stage2(LLaMA2, X, y, task_, a[-1][-1]))
    b.append(Self_Discover(LLaMA3, X, y, task_))
    d.append(Stage2(LLaMA3, X, y, task_, b[-1][-1]))

# answers1 = []
# answers2 = []
# answers3 = []
# for item in a:
#     answers1.append(item[0])
#     answers2.append(item[1])
#     answers3.append(item[2])
answers4 = []
answers5 = []
answers6 = []
for item in b:
    answers4.append(item[0])
    answers5.append(item[1])
    answers6.append(item[2])
df1 = pd.concat([X_train.reset_index(drop=True), df_miss[["label", "prediction"]]], axis = 1)
# df2 = pd.DataFrame({"LLaMA2_SELECT": answers1, "LLaMA2_ADAPT": answers2, "LLaMA2_IMPLEMENT": answers3, "LLaMA2_Answer": c,
#                     "LLaMA3_SELECT": answers4, "LLaMA3_ADAPT": answers5, "LLaMA3_IMPLEMENT": answers6, "LLaMA3_Answer": d})
df2 = pd.DataFrame({"LLaMA3_SELECT": answers4, "LLaMA3_ADAPT": answers5, "LLaMA3_IMPLEMENT": answers6, "LLaMA3_Answer": d})
df = pd.concat([df1, df2], axis = 1)
df.to_csv(f"./data/log/result_ver5_0607_mixed_miscalssification_ver_4_2.csv",index = False)

Generate Reasoning: 0it [00:00, ?it/s]

# Only Answer

In [ ]:
discovered_structure = """
 Sure! Here's a step-by-step reasoning plan in JSON format that can be adapted to classify the emotion in a given text using the five reasoning modules you provided:

{
"Reasoning Plan": {
"Module": {
"Critical Thinking": {
"Step 1": "Analyze the given data and evaluate the AI-derived emotion label. Question the assumptions behind the label and evaluate the evidence provided in the text. Consider alternative perspectives or viewpoints that may challenge the AI-derived label.",
"Step 2": "Evaluate the coherence and consistency of the text with respect to the AI-derived emotion label. Identify any inconsistencies or contradictions in the text that may suggest a different emotion label.",
"Step 3": "Compare the AI-derived emotion label with other possible emotion labels that may be more appropriate based on the content of the text."
},
"Creative Thinking": {
"Step 1": "Generate innovative and creative ideas for alternative emotion labels that may better capture the emotional tone of the text. Consider unconventional or unexpected emotion labels that may better reflect the complexity of the emotions expressed in the text.",
"Step 2": "Explore different artistic or cultural representations of emotions that may provide new insights into the emotional content of the text. For example, examine how different artistic styles or cultural traditions might represent the emotions expressed in the text.",
"Step 3": "Create new metaphors or analogies that might help to capture the complexity of the emotions expressed in the text."
},
"Reflective Thinking": {
"Step 1": "Take a step back and reflect on the problem of classifying the emotion in the given text. Identify any personal biases or assumptions that may be influencing your approach to the problem.",
"Step 2": "Examine the potential risks or drawbacks of the AI-derived emotion label. Consider any potential consequences of misclassifying the emotion in the text, or of overlooking important emotional details.",
"Step 3": "Consider alternative perspectives or viewpoints that may provide new insights into the emotional content of the text. For example, you might consider the emotions of the author or the intended audience of the text."
},
"Systems Thinking": {
"Step 1": "Consider the problem of classifying the emotion in the given text as part of a larger system of emotions and emotional experiences. Identify the interdependencies and feedback loops between different emotions and emotional experiences.",
"Step 2": "Evaluate the potential impact of the AI-derived emotion label on the larger system of emotions. Consider any potential consequences of labeling the emotion in the text, or of overlooking important emotional details.",
"Step 3": "Develop holistic solutions that address the system of emotions as a whole. Consider alternative approaches to classifying the emotion in the text that might better account for the complexity of emotional experiences."
},
"Risk Analysis": {
"Step 1": "Evaluate the potential risks and uncertainties associated with the AI-derived emotion label. Consider any potential consequences of misclassifying the emotion in the text, or of overlooking important emotional details.",
"Step 2": "Assess the likelihood of success or failure of the AI-derived emotion label. Consider any potential factors that may influence the accuracy of the label, such as the complexity of the emotions expressed in the text or the limitations of the AI algorithm.",
"Step 3": "Make informed decisions based on a balanced analysis of risks and benefits. Consider alternative approaches to classifying the emotion in the text that might better account for the complexity of emotional experiences while minimizing potential risks and uncertainties."
}
}

This reasoning plan can be adapted for each task by replacing the given text with the actual text data, and following the steps for each module. The final result will be a classification of the emotion in the text based on a holistic analysis of the data, taking into account the complexity of emotional experiences and the potential risks and uncertainties associated with the classification.
"""

In [ ]:
data = """
Where am I? I dont know where the fuck I am in life.

* im seventeen

* male

* I've initiated a sort of breaking up with a very dear friend last year

* an old family friend has specifically said they dont want anything to do with me any more

* I shut off cuminication with my father almost 2 years ago due to years of his alchohol abuse

* I live with my mother full time now and constantly argue with her about seemingly pointless things

* I cant stand the monotony that is school because i have no intrest in academics (which is unfortunelty the subjects i chose) and in turn i find more trouble at home due to the marks i get

I understand that maybe this is not some of the worst situations that could be happening to me but its just all so overwhelming. Nothing seems to really cheer me up any more. I can hardly laugh with my best mate of 17 years. We used to have so much fun.

I dont know what posting here will achieve or even if it will get any recognition.

I'm just so lost and scared.

Thanks.
"""

In [ ]:
task_ = task.replace("{data}", data).replace("{label}", "Depression")
answer = Stage2(LLaMA3, data, "Depression", task_, discovered_structure)

In [ ]:
print(answer)

1. Objectively analyze given data.

The text data reveals a sense of despair, hopelessness, and desperation. The individual is struggling to cope with various challenges in their life, including a recent breakup, family conflicts, and feelings of isolation. They express a sense of being lost and scared, and their emotions seem to be overwhelming them.

2. Compare the analysis results from Question 1 to AI's past classified label.

The AI-derived label is "Depression", which aligns with the analysis results from Question 1. The text data presents a clear picture of depression, with the individual expressing feelings of sadness, hopelessness, and despair.

3. Synthesize results from Questions 1 and 2 and write a final result for the given data.

Based on the analysis and comparison, the final result is:

"Depression"

4. Choose only one of the following that most closely resembles your analytics results.

"Depression"


In [ ]:
d=[]
for X, y in tqdm(zip(X_train.values, y_train), desc = "Generate Reasoning"):
    task_ = task.replace("{data}", X).replace("{label}", y)
    d.append(Stage2(LLaMA3, X, y, task_, discovered_structure))

df1 = pd.concat([X_train.reset_index(drop=True), df_miss[["label", "prediction"]]], axis = 1)
df2 = pd.DataFrame({"LLaMA3_Answer": d})
df = pd.concat([df1, df2], axis = 1)
df.to_csv(f"./data/log/result_ver5_0607_mixed_miscalssification_ver_2_4_3.csv",index = False)
print(*d, sep = "\n------------\n")

Generate Reasoning: 0it [00:00, ?it/s]

1. Objectively analyze given data.

The text data describes a housewife's mixed emotions about her life. She has built a happy home and is content with her role as a mother. However, she also feels despair due to the halt in her career development and feels envious when she sees her friends' careers. She worries about her husband's feelings and sometimes feels anxious while watching the baby. The text data reveals a complex emotional state, with both positive and negative emotions present.

2. Compare the analysis results from Question 1 to AI's past classified label.

The AI-derived classified label is "Happy", which does not accurately reflect the complexity of emotions present in the text data. The text data reveals a mix of emotions, including happiness, despair, envy, anxiety, and worry.

3. Synthesize results from Questions 1 and 2 and write a final result for the given data.

Based on the analysis, the final result is that the text data contains "Mixed Emotions". The text data d